In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import convnet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'convnet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=4, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 1)                 193       
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 19s]
val_loss: 22.32751703262329

Best val_loss So Far: 8.691648483276367
Total elapsed time: 00h 32m 06s


# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.2}
Number of parameters: 13553
========== Model 1 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 33353
========== Model 2 ==========
{'n_conv_blocks': 3, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 14145
========== Model 3 ==========
{'n_conv_blocks': 2, 'n_conv_layers'

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.2}
Number of parameters: 13553
Got score: 8.7540 (8.7732, 8.7348)
Got score: 8.5047 (8.6273, 8.3821)
Got score: 8.1975 (8.1673, 8.2276)
Got score: 8.6691 (8.6269, 8.7112)
Got score: 8.8543 (8.8641, 8.8445)


{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 33353
Got score: 8.9140 (8.9435, 8.8846)
Got score: 8.5821 (8.6081, 8.5561)
Got score: 8.3218 (8.3277, 8.3159)
Got score: 8.5666 (8.5596, 8.5736)
Got score: 9.3144 (8.9536, 9.6753)


{'n_conv_blocks': 3, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 14145
Got score: 16.2616 (23.6417, 8.8815)
Got score: 8.6464 (8.5600, 8.7328)
Got score: 8.3975 (8.3833, 8.4118)
Got score: 8.8268 (8.7511, 8.9024)
Got score: 9.4200 (9.8028, 9.0373)


{'n_conv_blocks': 2, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.0}
Number of parameters: 14897
Got score: 8.8773 (8.9006, 8.8541)
Got score: 8.6337 (8.6779, 8.5895)
Got score: 8.2739 (8.2477, 8.3002)
Got score: 8.7874 (8.8464, 8.7283)
Got score: 9.0191 (8.9956, 9.0426)


{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.2}
Number of parameters: 7377
Got score: 10.3299 (10.6246, 10.0352)
Got score: 20.1319 (31.5708, 8.6931)
Got score: 19.9867 (31.4067, 8.5667)
Got score: 30.3141 (22.4945, 38.1338)
Got score: 19.1370 (9.9611, 28.3128)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.60,0.23,13553
1,8.74,0.34,33353
2,10.31,2.99,14145
3,8.72,0.26,14897
4,19.98,6.33,7377
